# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [4]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [5]:
data.rename({"id":'race'}, axis='columns').head()
print(data.shape)

(4870, 65)


In [6]:
pd.crosstab(data.race, data.call, margins=True, margins_name="Total")

call,0.0,1.0,Total
race,,,
b,2278,157,2435
w,2200,235,2435
Total,4478,392,4870


#### 1. What test is appropriate for this problem? Does CLT apply?
* Binomial Test for Proportion
* CLT applies:
  *  n*p > 10
  *  n*(1-p) > 10
  *  random sample
  *  n is less than 10% of population

#### 2. What are the null and alternate hypotheses?
Binomial Test for Proportions:
* Ho: A name sounding black or white has no effect on getting a person getting a call back for a job.
* Ha: The difference in proportions is significant: The proportion of people with black-sounding names receiving a call back is less than the proportion of the population with black-sounding names. 

#### 3. Compute margin of error, confidence interval, and p-value. 
Try using both the bootstrapping and the frequentist statistical approaches.

In [7]:
# Binomial Hypothesis Test
alpha = .05 

w = data[data.race=='w']
b = data[data.race=='b']
w_all = len(w)
b_all = len(b)
bc_wins = np.sum(b.call)
wc_wins = np.sum(w.call)

# number of successful call backs total
n = np.sum(data.call)

# sample proportion of black call backs
prop_emp = bc_wins / n

# ho proportion of black call backs determined by total population
prop_ho = b_all / (w_all+b_all)

# Standard error
SE = np.sqrt((prop_ho*(1-prop_ho))/n)

# Z and P value
z = (prop_emp - prop_ho) / SE
pval = st.norm.sf(abs(z))*2 

print("P Value: %.10f is less than %f, so we reject Ho." % (pval, alpha)) if pval < alpha \
else print("P Value: %.10f is more than %f, so we fail to reject Ho." % (pval, alpha))

P Value: 0.0000816191 is less than 0.050000, so we reject Ho.


In [8]:
# Confidence interval for true population proportion

# Z score for 95% confidence
z = st.norm.ppf(1-(1-0.95)/2)

#confidence interval 
low, high = prop_emp - z*SE, prop_emp + z*SE

print("The margin of error is %.3f." % abs(z*SE))
print("With 95%% confidence, the proportion of people with black-sounding names who get call backs is between %.3f and %.3f." \
      % (low, high))

The margin of error is 0.049.
With 95% confidence, the proportion of people with black-sounding names who get call backs is between 0.351 and 0.450.


###### Another method:
* Ho: The proportion of callbacks received in populations with white-sounding names is equal to the proportion of callbacks received in populations with black-sounding names.
* Ha: There is a statistically significantly difference in proportion: The proportion of callbacks received in populations with white-sounding names greater than the proportion of callbacks received in populations with black-sounding names.

In [9]:
# difference of proportions between 2 populations

w = data[data.race=='w']
b = data[data.race=='b']

#functions:

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2


def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

#  High diff_fracs means higher proportion of white wins 
def diff_frac(b, w):
    frac_b = np.sum(b) / len(b)
    frac_w = np.sum(w) / len(w)
    return frac_w - frac_b

diff_frac_obs = diff_frac(b.call, w.call)

perm_reps = draw_perm_reps(b.call, w.call, diff_frac, 10000)

p = np.sum(perm_reps > diff_frac_obs) / 10000

print("Difference in observed proportions:", diff_frac_obs)
print("Examples of difference in proportions from permutations:", perm_reps[:10])
print("P value=%.3f is less than %.3f, so we reject Ho." % (p, alpha)) if p < alpha \
else print("P value=%.3f is more than %.3f, so we fail to reject Ho. " % (p, alpha))

Difference in observed proportions: 0.032032854209445585
Examples of difference in proportions from permutations: [-0.00164271  0.00082136 -0.00574949  0.00164271 -0.00657084  0.00246407
  0.00246407  0.00246407 -0.01232033  0.00657084]
P value=0.000 is less than 0.050, so we reject Ho.


#### 4. Write a story describing the statistical significance in the context or the original problem.
Bill H.R.40 - Commission to Study and Develop Reparation Proposals for African-Americans Act was introduced to the house in 01/03/2019 . <br>
<br>
Purpose: "To address the fundamental injustice, cruelty, brutality, and inhumanity of slavery in the United States and the 13 American colonies between 1619 and 1865 and to establish a commission to study and consider a national apology and proposal for reparations for the institution of slavery, its subsequent de jure and de facto racial and economic discrimination against African-Americans, and the impact of these forces on living African-Americans, to make recommendations to the Congress on appropriate remedies, and for other purposes." <br>
<br>
If this bill passes, scientific studies can determine how to best undertake this issue. Knowing that the likelihood of getting a job is significantly impacted by perceived race can support legislation in supporting equal access to employment. 

#### 5. Does your analysis mean that race/name is the most important factor in callback success? 
Why or why not? If not, how would you amend your analysis? 
*  No, this does not signify that race/name is the most important factor in callback successes. In order to determine which factors are most important, we would need to determine the significance of each other variable on employment. I could use chi-squared tests for other variables, as they are categorical, to test for significance of association. 